Based on:

https://github.com/Deepthi10/Intrusion-Detection-using-Machine-Learning-on-NSL--KDD-dataset/blob/master/IDS.ipynb

Pre-req:

create under data the dir: processed_nslkdd_v2

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
import warnings
import os

warnings.filterwarnings("ignore")

In [14]:
DATA_DIR = '../data'
MODEL_DIR = '../models'
MODEL_NAME = os.path.join(MODEL_DIR, 'model_v1.pkl')

FILE_PATH_TRAIN = os.path.join(DATA_DIR, 'raw/KDDTrain+_20Percent.csv')
FILE_PATH_TEST = os.path.join(DATA_DIR, 'raw/KDDTest-21.csv')

FILE_PATH_TRAIN_FEATURES = os.path.join(DATA_DIR, 'processed_nslkdd_v2', 'train/train_features.csv')
FILE_PATH_TRAIN_LABLES = os.path.join(DATA_DIR, 'processed_nslkdd_v2', 'train/train_labels.csv')

FILE_PATH_TEST_FEATURES = os.path.join(DATA_DIR, 'processed_nslkdd_v2', 'test/test_features.csv')
FILE_PATH_TEST_LABLES = os.path.join(DATA_DIR, 'processed_nslkdd_v2', 'test/test_labels.csv')

In [15]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label", "difficulty_level"]

In [24]:
train_df = pd.read_csv(FILE_PATH_TRAIN)
test_df = pd.read_csv(FILE_PATH_TEST)

train_df = pd.read_csv(FILE_PATH_TRAIN)
test_df = pd.read_csv(FILE_PATH_TEST)

train_df.drop('difficulty_level', axis=1, inplace=True)
test_df.drop('difficulty_level', axis=1, inplace=True)


In [25]:
train_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [26]:
test_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,13,tcp,telnet,SF,118,2425,0,0,0,0,...,10,0.38,0.12,0.04,0.0,0.00,0.00,0.12,0.3,guess_passwd
1,0,udp,private,SF,44,0,0,0,0,0,...,254,1.00,0.01,0.01,0.0,0.00,0.00,0.00,0.0,snmpguess
2,0,tcp,telnet,S3,0,44,0,0,0,0,...,79,0.31,0.61,0.00,0.0,0.21,0.68,0.60,0.0,processtable
3,0,udp,private,SF,53,55,0,0,0,0,...,255,1.00,0.00,0.87,0.0,0.00,0.00,0.00,0.0,normal
4,0,tcp,private,SH,0,0,0,0,0,0,...,1,0.06,1.00,1.00,0.0,1.00,1.00,0.00,0.0,nmap


In [28]:
print('Label distribution Training set:')
print(train_df['class'].value_counts())
print()
print('Label distribution Test set:')
print(test_df['class'].value_counts())

Label distribution Training set:
normal             13449
neptune             8282
ipsweep              710
satan                691
portsweep            587
smurf                529
nmap                 301
back                 196
teardrop             188
warezclient          181
pod                   38
guess_passwd          10
warezmaster            7
buffer_overflow        6
imap                   5
rootkit                4
multihop               2
phf                    2
ftp_write              1
land                   1
loadmodule             1
spy                    1
Name: class, dtype: int64

Label distribution Test set:
normal             2152
neptune            1579
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               727
processtable        685
smurf               627
back                359
snmpguess           331
saint               309
mailbomb            293
snmpgetattack       178
portsweep           156
i

In [29]:
# insert code to get a list of categorical columns into a variable, categorical_columns
categorical_columns=['protocol_type', 'service', 'flag'] 
 # Get the categorical values into a 2D numpy array
train_categorical_values_df = train_df[categorical_columns]
test_categorical_values_df = test_df[categorical_columns]

In [30]:
train_categorical_values_df.head()


,protocol_type,service,flag
0,tcp,ftp_data,SF
1,udp,other,SF
2,tcp,private,S0
3,tcp,http,SF
4,tcp,http,SF


# Dummy Columns


In [31]:
# protocol type
unique_protocol=sorted(train_df.protocol_type.unique())
unique_protocol
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
# service
unique_service=sorted(train_df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
# flag
unique_flag=sorted(train_df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
# put together
dumcols=unique_protocol2 + unique_service2 + unique_flag2
print(dumcols)

#do same for test set
unique_service_test=sorted(test_df.service.unique())
unique_service2_test=[string2 + x for x in unique_service_test]
testdumcols=unique_protocol2 + unique_service2_test + unique_flag2

['Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp', 'service_IRC', 'service_X11', 'service_Z39_50', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard', 'service_domain', 'service_domain_u', 'service_echo', 'service_eco_i', 'service_ecr_i', 'service_efs', 'service_exec', 'service_finger', 'service_ftp', 'service_ftp_data', 'service_gopher', 'service_hostnames', 'service_http', 'service_http_443', 'service_http_8001', 'service_imap4', 'service_iso_tsap', 'service_klogin', 'service_kshell', 'service_ldap', 'service_link', 'service_login', 'service_mtp', 'service_name', 'service_netbios_dgm', 'service_netbios_ns', 'service_netbios_ssn', 'service_netstat', 'service_nnsp', 'service_nntp', 'service_ntp_u', 'service_other', 'service_pm_dump', 'service_pop_2', 'service_pop_3', 'service_printer', 'service_private', 'service_red_i', 'service_remote_job', 'service_rje', 'service_shell', 'service_smtp', 'service_sql

# Transform categorical features into numbers using LabelEncoder()

In [32]:
#Transform categorical features into numbers using LabelEncoder()
train_categorical_values_enc_df = train_categorical_values_df.apply(LabelEncoder().fit_transform)
print(train_categorical_values_enc_df.head())
# test set
test_categorical_values_enc_df = test_categorical_values_df.apply(LabelEncoder().fit_transform)

   protocol_type  service  flag
0              1       19     9
1              2       41     9
2              1       46     5
3              1       22     9
4              1       22     9


# One-Hot-Encoding


In [33]:
#One-Hot-Encoding
ohe = OneHotEncoder()
train_categorical_values_ohe_df = ohe.fit_transform(train_categorical_values_enc_df)
train_cat_data_df = pd.DataFrame(train_categorical_values_ohe_df.toarray(),columns=dumcols)
# test set
test_categorical_values_ohe_df = ohe.fit_transform(test_categorical_values_enc_df)
test_cat_data_df = pd.DataFrame(test_categorical_values_ohe_df.toarray(),columns=testdumcols)

In [34]:
train_cat_data_df.head()


,Protocol_type_icmp,Protocol_type_tcp,Protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,service_courier,service_csnet_ns,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Add 6 missing categories from train set to test set


In [35]:
trainservice = train_df['service'].tolist()
testservice = test_df['service'].tolist()
difference = list(set(trainservice) - set(testservice))
string = 'service_'
difference = [string + x for x in difference]
difference

['service_rje',
 'service_urh_i',
 'service_http_8001',
 'service_printer',
 'service_red_i']

In [36]:
for col in difference:
    test_cat_data_df[col] = 0

test_cat_data_df.shape

(11850, 81)

# Join encoded categorical dataframe with the non-categorical dataframe


In [37]:
new_train_df=train_df.join(train_cat_data_df)
new_train_df.drop('flag', axis=1, inplace=True)
new_train_df.drop('protocol_type', axis=1, inplace=True)
new_train_df.drop('service', axis=1, inplace=True)
# test data
new_test_df=test_df.join(test_cat_data_df)
new_test_df.drop('flag', axis=1, inplace=True)
new_test_df.drop('protocol_type', axis=1, inplace=True)
new_test_df.drop('service', axis=1, inplace=True)
print(new_train_df.shape)
print(new_test_df.shape)

(25192, 119)
(11850, 120)


# Rename every attack label: 0=normal, 1=DoS, 1=Probe, 1=R2L and 1=U2R.


In [38]:
#take label column
label_train_df = new_train_df['class']
label_test_df = new_test_df['class']

#change the label column
new_train_label_df = label_train_df.replace({'normal':0,'neptune':1,'back':1,'land':1,'pod':1,'smurf':1,'teardrop':1,'mailbomb':1,'apache1':1,'processtable':1,'udpstorm':1,'worm':1,
'ipsweep':1,'nmap':1,'portsweep':1,'satan':1,'mscan':1,'saint':1
,'ftp_write':1,'guess_passwd':1,'imap':1,'multihop':1,'phf':1,'spy':1,'warezclient':1,'warezmaster':1,'sendmail':1,'named':1,'snmpgetattack':1,'snmpguess':1,'xlock':1,'xsnoop':1,'httptunnel':1,
'buffer_overflow':1,'loadmodule':1,'perl':1,'rootkit':1,'ps':1,'sqlattack':1,'xterm':1, 'apache2':1})
new_test_label_df_test=label_test_df.replace({'normal':0,'neptune':1,'back':1,'land':1,'pod':1,'smurf':1,'teardrop':1,'mailbomb':1,'apache1':1,'processtable':1,'udpstorm':1,'worm':1,
'ipsweep':1,'nmap':1,'portsweep':1,'satan':1,'mscan':1,'saint':1
,'ftp_write':1,'guess_passwd':1,'imap':1,'multihop':1,'phf':1,'spy':1,'warezclient':1,'warezmaster':1,'sendmail':1,'named':1,'snmpgetattack':1,'snmpguess':1,'xlock':1,'xsnoop':1,'httptunnel':1,
'buffer_overflow':1,'loadmodule':1,'perl':1,'rootkit':1,'ps':1,'sqlattack':1,'xterm':1, 'apache2':1})

# put the new label column back
new_train_df['class'] = new_train_label_df
new_test_df['class'] = new_test_label_df_test
print(new_train_df['class'].head())

0    0
1    0
2    1
3    0
4    0
Name: class, dtype: int64


In [39]:
X_train = new_train_df.drop('class',1)
Y_train = new_train_df['class']

X_test = new_test_df.drop('class',1)
Y_test = new_test_df['class']

# Use StandardScaler() to scale the dataframes


In [40]:
scaler = preprocessing.MinMaxScaler()

X_train = pd.DataFrame(scaler.fit(X_train).transform(X_train))
X_test = pd.DataFrame(scaler.fit(X_test).transform(X_test))

In [41]:
print(X_train.std(axis=0))

0      0.062679
1      0.006316
2      0.017244
3      0.008910
4      0.086740
         ...   
113    0.059001
114    0.028861
115    0.024395
116    0.491026
117    0.041280
Length: 118, dtype: float64


In [42]:
# Save the results
X_train.to_csv(os.path.join(DATA_DIR, 'processed_nslkdd_v2', 'train/train_features.csv'), index=False, header=None)
X_test.to_csv(os.path.join(DATA_DIR, 'processed_nslkdd_v2', 'test/test_features.csv'), index=False, header=None)

Y_train.to_csv(os.path.join(DATA_DIR, 'processed_nslkdd_v2', 'train/train_labels.csv'), index=False, header=None)
Y_test.to_csv(os.path.join(DATA_DIR, 'processed_nslkdd_v2', 'test/test_labels.csv'), index=False, header=None)

,0,1,2,3,4,5,6,7,8,9,...,108,109,110,111,112,113,114,115,116,117
0,0.0,1.286320e-06,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,3.824902e-07,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,6.077927e-07,0.001583,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,5.213394e-07,0.000082,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25187,0.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25188,0.0,8.750119e-07,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
25189,0.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25190,0.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
